## Metro BRT

Take a look at Metro's J Line and pick out stops that run on the street but have bus lanes. These will be included as BRT.

We will still use a list of stops to exclude, pare down the street running stops to stops that do not occur on bus lanes.

In [ ]:
import folium
import geopandas as gpd
import pandas as pd

from segment_speed_utils import helpers
from update_vars import analysis_date, TEMP_GCS

In [ ]:
from shared_utils import rt_utils

metro = "LA Metro Bus Schedule"

brt_stops = gpd.read_parquet(
    f"{rt_utils.GCS_FILE_PATH}temp/brt_stops.parquet",
    filters = [[("name", "==", metro)]]
)

In [ ]:
trips = helpers.import_scheduled_trips(
    analysis_date,
    filters = [[("name", "==", metro),
                ("route_desc", "in", ["METRO G LINE", "METRO J LINE"])
               ]],
    columns = ["shape_array_key", "route_id", "route_desc", "direction_id"],
    get_pandas = True
).drop_duplicates(subset=["route_desc", "direction_id"])

shapes = helpers.import_scheduled_shapes(
    analysis_date,
    filters = [[("shape_array_key", "in", trips.shape_array_key)]],
    columns = ["shape_array_key", "geometry"],
    get_pandas = True
)

In [ ]:
m = brt_stops[
        (brt_stops.name==metro) & 
        (brt_stops.route_id.str.contains("910"))
    ].explore("route_id", name = "stops", tiles = "CartoDB Positron") 

m = shapes.explore(
    "shape_array_key", 
    m=m, name = "shape", color="orange"
)

folium.LayerControl().add_to(m)
m

In [ ]:
metro_j_exclude = [
    # Grand DTLA - no bus lanes
    '13560', '13561',
    # 1st St DTLA
    '5377', '15612',
    # Spring St DTLA on grand park side...no bus lanes?
    '12416',    
    # Figueroa St Gardena/Carson before it gets onto 110
    '65300038', '65300039',
    # Beacon St Harbor Gateway area
    '378', '3124', '3153', '2603', 
    # Pacific Ave Harbor Gateway area
    '3821', '12304', '13817', '5408', '5410', '5411', 
    '13802', '5395', '13803', '5396', '13804', '5397',
    '13805', '141012',
]

In [ ]:
m2 = brt_stops[
    (brt_stops.name==metro) & 
    (brt_stops.route_id.str.contains("910")) & 
    ~(brt_stops.stop_id.isin(metro_j_exclude))
].explore("route_id", name = "stops", tiles = "CartoDB Positron") 

m2 = shapes.explore(
    "shape_array_key", 
    m=m2, name = "shape", color="orange"
)

folium.LayerControl().add_to(m2)
m2